In [116]:
import requests
import os
import numpy as np

cookies = {
  "session": os.getenv('AOC_SESSION_TOKEN'),
}

def get_data_for_day(day, test=False):
    append = '-test' if test else ''
    filepath = f'data/day-{day}{append}.txt'
    data = None
    if os.path.isfile(filepath):
        data = open(filepath, 'r').read()
    else:
        req = requests.get(f"https://adventofcode.com/2021/day/{day}/input", cookies=cookies)
        data = req.content.decode('utf-8')
        f = open(filepath, "a")
        f.write(data)
        f.close()
    
    return data.split('\n')[0:-1]

# Day 1

In [184]:
altitudes = [int(i) for i in get_data_for_day(1)]

counter = 0
counter_b = 0
for index in range(0, len(altitudes)):
    if index >= 1 and altitudes[index] > altitudes[index-1]:
        counter += 1
    
    
    last_window = sum(altitudes[index-3:index])
    current_window = sum(altitudes[index-2:index+1])
#     print(index, altitudes[index], current_window, last_window, current_window > last_window)
    if index >= 3:
        if current_window > last_window:
            counter_b += 1
    
print(len(altitudes), counter, counter_b)

2000 1559 1600


# Day 2

In [69]:
commands = [tuple(i.split(' ')) for i in get_data_for_day(2)]

# Part A
horizontal = 0
depth = 0
for command in commands:
    direction = command[0]
    value = int(command[1])
    if direction in ['forward']:
        horizontal += value
    if direction in ['up']:
        depth -= value
    if direction in ['down']:
        depth += value
        
print(horizontal*depth)

# Part B
horizontal_b = 0
depth_b = 0
aim = 0
for command in commands:
    direction = command[0]
    value = int(command[1])
    if direction in ['forward']:
        horizontal_b += value
        depth_b += value*aim
    if direction in ['up']:
        aim -= value
    if direction in ['down']:
        aim += value
        
print(horizontal_b*depth_b)

1989014
2006917119


# Day 3

In [85]:
report = [i for i in get_data_for_day(3)]

In [86]:
report_rows = len(report)

counters = [0] * len(report[0])
for row in report:
    for index, item in enumerate(row):
        bit = int(item)
        counters[index] += bit
        
gamma = int(''.join([str(int(c > report_rows / 2)) for c in counters]), 2)
epsilon = int(''.join([str(int(c < report_rows / 2)) for c in counters]), 2)
print('Part A:', len(report[0]), counters, gamma, epsilon, gamma * epsilon)

Part A: 12 [504, 496, 497, 525, 517, 528, 519, 497, 498, 498, 503, 502] 2531 1564 3958484


In [97]:
report_test = ['00100', '11110', '10110', '10111', '10101', '01111', '00111', '11100', '10000', '11001', '00010', '01010']

def get_remaining_rating(remaining, least, most):
    index = 0
    while len(remaining) > 1:
        zeros_ones = [0, 0]
        for row in remaining:
            zeros_ones[int(row[index])] += 1

        if zeros_ones[0] > zeros_ones[1]:
            remaining = [r for r in remaining if r[index] == most]
        else:
            remaining = [r for r in remaining if r[index] == least]
        index += 1
    return int(remaining[0], 2)

print('Part B:', get_remaining_rating(report, '0', '1') * get_remaining_rating(report, '1', '0'))


Part B: 1613181


# Day 4

In [273]:
day_4_data = [i for i in get_data_for_day(4, False)]

class Board:
    def __init__(self, data):
        self.data = data
        self.items = np.array(data, tuple)
        self.skip = False
        for x, row in enumerate(data):
            for y, value in enumerate(row):
                self.items[x][y] = (int(value), False)
        
    def get_row(self, x):
        return self.items[x]
    
    def get_column(self, x):
        return [self.items[t][x] for t in range(5)]

    def check_number(self, number):
        for x, row in enumerate(self.items):
            for y, value in enumerate(row):
                if value[0] == number:
                    self.items[x][y] = (int(number), True)
                
    def has_bingo(self):
        for r in range(5):
            bingo_row = len([i for i in range(5) if self.get_row(r)[i][1]]) == 5
            bingo_column = len([i for i in range(5) if self.get_column(r)[i][1]]) == 5
            if bingo_row or bingo_column:
                return True
        return False
    
    def get_sum_unmarked(self):
        sums = 0
        for x, row in enumerate(self.items):
            for y, value in enumerate(row):
                if not value[1]:
                    sums += value[0]
                
        return sums

board_data_raw = day_4_data[2:]

boards = []
board_data = []
for line in board_data_raw:
    row = line.split()
    if len(row) == 5:
        board_data.append(row)
    if len(board_data) == 5:
        boards.append(Board(board_data))
        board_data = []
    

bingo_numbers = [int(n) for n in day_4_data[0].split(',')]

last_board = None
def play_bingo():
    first = True
    for bingo_number in bingo_numbers:
        for board in boards:
            board.check_number(bingo_number)

            if board.has_bingo() and not board.skip:
                last_board = board
                if first:
    #                 print(board.items)
                    print(board.get_sum_unmarked(), bingo_number)
                    print(board.get_sum_unmarked() * bingo_number)
                    print(board.items)
                    first = False
                board.skip = True
        
        not_bingo_boards = [board for board in boards if not board.has_bingo()]
        if len(not_bingo_boards) == 0:
            print(last_board.get_sum_unmarked(), bingo_number)
            print(last_board.get_sum_unmarked() * bingo_number)
            print(last_board.items)
            return

play_bingo()



784 51
39984
[[(54, False) (97, False) (46, False) (33, False) (90, False)]
 [(99, True) (93, True) (22, True) (0, True) (51, True)]
 [(83, False) (53, False) (34, False) (29, False) (38, True)]
 [(35, False) (65, True) (80, True) (82, False) (9, False)]
 [(56, False) (30, True) (19, False) (49, False) (15, False)]]
116 73
8468
[[(11, False) (69, True) (4, True) (6, True) (23, True)]
 [(38, True) (47, False) (16, False) (99, True) (96, True)]
 [(7, True) (13, True) (40, True) (41, False) (78, True)]
 [(12, True) (5, True) (1, False) (18, True) (88, True)]
 [(20, True) (42, True) (10, True) (82, True) (73, True)]]


# Day 5

In [346]:
import re

all_x = []
all_y = []

day_5_data = [i for i in get_data_for_day(5, False)]
for row in day_5_data:
    prog = re.compile('^(\d+),(\d+) -> (\d+),(\d+)$')
    found = prog.match(row)
    x1 = int(found.group(1))
    y1 = int(found.group(2))
    x2 = int(found.group(3))
    y2 = int(found.group(4))
    all_x.append(x1)
    all_x.append(x2)
    all_y.append(y1)
    all_y.append(y2)
    
grid = np.zeros(((max(all_x) + 1), (max(all_y)) + 1), dtype=int)

for row in day_5_data:
    prog = re.compile('^(\d+),(\d+) -> (\d+),(\d+)$')
    found = prog.match(row)
    x1 = int(found.group(1))
    y1 = int(found.group(2))
    x2 = int(found.group(3))
    y2 = int(found.group(4))
    if x1 == x2:
        for y in range(min([y1, y2]), (max([y1, y2]) + 1)):
            grid[y][x1] += 1
    elif y1 == y2:
        for x in range(min([x1, x2]), (max([x1, x2]) + 1)):
            grid[y1][x] += 1
    else:
        for i, y in enumerate(range(y1, (y2 + 1) if y1 < y2 else (y2 - 1), 1 if y1 < y2 else -1)):
            for j, x in enumerate(range(x1, (x2 + 1) if x1 < x2 else (x2 - 1), 1 if x1 < x2 else -1)):
                if i == j:
                    grid[y][x] += 1
                
print(grid)

total = 0
for x, row in enumerate(grid):
    for y, value in enumerate(row):
        if value > 1:
            total += 1
print(total)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 1 2 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
19676


# Day 6

In [2]:
# Part A
day_6_data = list(map(int, get_data_for_day(6, False)[0].split(',')))
# print(day_6_data)

def get_fish(initial_data, days):
    daily_data = initial_data
    for day in range(days):
        zeros = len([d for d in daily_data if d == 0])
        daily_data = [6 if d == 0 else d - 1 for d in daily_data]
        for z in range(zeros):
            daily_data.append(8)
    return daily_data

print(len(get_fish(day_6_data, 80)))

374994


In [3]:
# Part B
sums = 0
day_map = {}
for day in range(9):
    day_map[day] = get_fish([day], 128)
    
for fish_lifes in day_6_data:
    for fish_life in day_map[fish_lifes]:
        sums += len(day_map[fish_life])

print(sums)

1686252324092


# Day 7

In [28]:
# Part A
import statistics

day_7_data = list(map(int, get_data_for_day(7, False)[0].split(',')))
median = int(statistics.median(day_7_data))

fuel_a = [abs(median-move) for move in day_7_data]
print(f"Part A: {sum(fuel_a)}")

avg = round(sum(day_7_data) / len(day_7_data))
min_sum = float('Infinity')
for x in range(min(day_7_data), max(day_7_data)):
    fuel_b = [(abs(x-move) * (abs(x-move) + 1) / 2) for move in day_7_data]
    min_sum = min(int(sum(fuel_b)), min_sum)
print(f"Part B: {min_sum}")

Part A: 337833
Part B: 96678050


# Day 8

In [117]:
day_8_data = [row.split(' | ') for row in list(get_data_for_day(8, False))]

count_per_digit = {
    0: 6,
    1: 2,
    2: 5,
    3: 5,
    4: 4,
    5: 5,
    6: 6,
    7: 3,
    8: 7,
    9: 6,
}

counter = 0
for groups in day_8_data:
    counter += len([diget for diget in groups[1].split() if len(diget) in list(count_per_digit.values())])

print(f"Part A: {counter}")

def sort_digit_letters(digit_letters):
    return [''.join(sorted(list(d))) for d in digit_letters]

def deduct_digets(all_digits):
    digits = ['' for i in range(10)]
    for item in all_digits:
        if len(item) == 2:
            digits[1] = item
        elif len(item) == 3:
            digits[7] = item
        elif len(item) == 4:
            digits[4] = item
        elif len(item) == 7:
            digits[8] = item

    for item in all_digits:
        if len(item) == 5 and len([l for l in item if l not in digits[7]]) == 2:
            digits[3] = item

    for item in all_digits:
        if len(item) == 6 and len([l for l in item if l not in digits[3]]) == 1:
            digits[9] = item

    for item in all_digits:
        if len(item) == 5 and len([l for l in item if l not in digits[9]]) == 1:
            digits[2] = item

    for item in all_digits:
        if len(item) == 5 and item not in digits:
            digits[5] = item

    for item in all_digits:
        if len(item) == 6 and len([l for l in digits[3] if l not in item]) == 1 and len([l for l in digits[1] if l not in item]) == 1:
            digits[6] = item

    for item in all_digits:
        if len(item) == 6 and item not in digits:
            digits[0] = item

    sorted_digits = sort_digit_letters(digits)
    return sorted_digits
    
code_sum = 0
for row in day_8_data:
    shuffled_digits = row[0].split(' ')
    code = row[1].split(' ')
    deducted = deduct_digets(shuffled_digits)
    decoded_code = int(''.join([str(deducted.index(d)) for d in sort_digit_letters(code)]))
    code_sum += decoded_code

print(f"Part B: {code_sum}")

Part A: 800
Part B: 1011823
